N.b. This requries a local large language model to be running. Its rquired that users download OLLAMA before they can do anything https://github.com/ollama/ollama 

The Ollama server also needs to be running for this code to work

I.e. #ollama run llama3.2

In [154]:
from itertools import combinations
from pyvis.network import Network
import networkx as nx
import numpy as np
import pandas as pd
import re
import json
import copy
import nltk
from nltk.tokenize import sent_tokenize
from google import genai
import ollama
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from nltk.tokenize import word_tokenize
nltk.download("punkt")


# 🔹 Replace with your actual API key
API_KEY = "sk-fc3beab1e18e4190bcf1b4a996733a5a"
WEBUI_URL = "http://localhost:3000/api/chat/completions"

[nltk_data] Downloading package punkt to /Users/svztg/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [126]:
datasets = []
publications = []
tools = []
with open ('./data/datasets.json') as f:
    datasets = json.load(f)
with open ('./data/publications.json') as f:
    publications = json.load(f)
with open ('./data/tools.json') as f:
    tools = json.load(f)

len(datasets), len(publications), len(tools)

(1059, 3199, 243)

In [127]:
titles = [x['metadata']['summary']['title'] for x in datasets]
titles[:5]

['Cystic Fibrosis Patient Microbiology Cultures',
 'Cystic Fibrosis Patient Liver Enzyme',
 'Cystic Fibrosis Patient Annual Review Encounters',
 'Cystic Fibrosis Patient Transplants',
 'Cystic Fibrosis Patient Sweat Tests']

In [128]:
descriptions = [x['metadata']['summary']['description'] for x in datasets]
descriptions[:5]

["The UK CF Registry is a centralised database of all 60 CF centres across the UK. Data are manually entered in calendar years by CF clinical teams for the 99% of people with a diagnosis of CF who consent to their data being donated to the Registry. Data are entered onto a secure web-portal. For more information please see www.cysticfibrosis.org.uk/registry and 'Data Resource Profile: The UK CF Registry' published in the International Journal of Epidemiology (2018 Feb 1;47(1)9-10e).",
 "The UK CF Registry is a centralised database of all 60 CF centres across the UK. Data are manually entered in calendar years by CF clinical teams for the 99% of people with a diagnosis of CF who consent to their data being donated to the Registry. Data are entered onto a secure web-portal. For more information please see www.cysticfibrosis.org.uk/registry and 'Data Resource Profile: The UK CF Registry' published in the International Journal of Epidemiology (2018 Feb 1;47(1)9-10e).",
 "The UK CF Registry

In [172]:
paper_abstracts = [x['abstract'] for x in publications]
paper_doi = [x['paper_doi'] for x in publications]
paper_abstracts[1]
abstract = paper_abstracts[50]

print(abstract)

<h4>Background</h4>Clinical trials generally each collect their own data despite routinely collected health data (RCHD) increasing in quality and breadth. Our aim is to quantify UK-based randomised controlled trials (RCTs) accessing RCHD for participant data, characterise how these data are used and thereby recommend how more trials could use RCHD.<h4>Methods</h4>We conducted a systematic review of RCTs accessing RCHD from at least one registry in the UK between 2013 and 2018 for the purposes of informing or supplementing participant data. A list of all registries holding RCHD in the UK was compiled. In cases where registries published release registers, these were searched for RCTs accessing RCHD. Where no release register was available, registries were contacted to request a list of RCTs. For each identified RCT, information was collected from all publicly available sources (release registers, websites, protocol etc.). The search and data extraction were undertaken between January an

In [155]:
# 🔹 List available models to check correct name
def list_models():
    headers = {"Authorization": f"Bearer {API_KEY}"}
    response = requests.get("http://localhost:3000/api/models", headers=headers)
    
    try:
        models = response.json()
        print("Available Models:", models)
        return models
    except requests.exceptions.RequestException as e:
        return f"Error fetching models: {e}"

# 🔹 Get available models and use a correct one
available_models = list_models()

print(available_models)

Available Models: {'data': [{'id': 'ChatGPT.o1-mini-2024-09-12', 'object': 'model', 'created': 1725648979, 'owned_by': 'openai', 'name': 'ChatGPT.o1-mini-2024-09-12', 'openai': {'id': 'ChatGPT.o1-mini-2024-09-12', 'object': 'model', 'created': 1725648979, 'owned_by': 'system'}, 'urlIdx': 0, 'actions': []}, {'id': 'ChatGPT.o1-preview-2024-09-12', 'object': 'model', 'created': 1725648865, 'owned_by': 'openai', 'name': 'ChatGPT.o1-preview-2024-09-12', 'openai': {'id': 'ChatGPT.o1-preview-2024-09-12', 'object': 'model', 'created': 1725648865, 'owned_by': 'system'}, 'urlIdx': 0, 'actions': []}, {'id': 'ChatGPT.o1-mini', 'object': 'model', 'created': 1725649008, 'owned_by': 'openai', 'name': 'ChatGPT.o1-mini', 'openai': {'id': 'ChatGPT.o1-mini', 'object': 'model', 'created': 1725649008, 'owned_by': 'system'}, 'urlIdx': 0, 'actions': []}, {'id': 'ChatGPT.o1-preview', 'object': 'model', 'created': 1725648897, 'owned_by': 'openai', 'name': 'ChatGPT.o1-preview', 'openai': {'id': 'ChatGPT.o1-prev

In [190]:
import requests

# 🔹 Replace with your actual API key
API_KEY = "sk-fc3beab1e18e4190bcf1b4a996733a5a"
WEBUI_URL = "http://localhost:3000/api/chat/completions"

def can_you_find_a_dataset(description, model):
    prompt = (
        "The text below is from a research publication. Can you extract the names of any datasets mentioned? "
        "Return a **comma-separated list** of dataset names. If none are found, return 'none'.\n\n"
        + description
    )

    headers = {
        "Authorization": f"Bearer {API_KEY}",
        "Content-Type": "application/json",
    }

    payload = {
        "model": model,
        "messages": [{"role": "user", "content": prompt}]
    }

    response = requests.post(WEBUI_URL, headers=headers, json=payload)
    json_response = response.text
        
    return json_response

# 🔹 Test with a sample abstract
#abstract = "This study uses the UK Biobank and Genomics England dataset."
dataset_names = can_you_find_a_dataset(abstract, "Gemma2:latest")
print(dataset_names)


{"id":"Gemma2:latest-0cbebc83-4b1a-4d47-b62a-af2e55f30744","created":1739460029,"model":"Gemma2:latest","choices":[{"index":0,"logprobs":null,"finish_reason":"stop","message":{"content":"mortality, hospital visits \n","role":"assistant"}}],"object":"chat.completion","usage":{"response_token/s":2.08,"prompt_token/s":11.68,"total_duration":49755775200,"load_duration":85127600,"prompt_eval_count":535,"prompt_eval_duration":45820000000,"eval_count":8,"eval_duration":3848000000,"approximate_total":"0h0m49s"}}
